In [37]:
#import all of the needed libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

#important libraries for NLP
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')

from bert_serving.client import BertClient
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
from nltk.corpus import stopwords
from gensim.models import Word2Vec
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
import gensim
from gensim import corpora

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\603766\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
df=pd.read_csv('../data/metadata.csv')

In [39]:
df.shape

(45774, 17)

In [40]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [41]:
#data cleaning dedup on abstract
#how orignial the abstract is
#based on the references and resources the author created the 
#how similar the paper is to the connecting points
#originality score

In [42]:
abstracts=df['abstract']

In [43]:
#Clean the documents
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop1 = set(stopwords.words('english'))
stop2 = set(stopwords.words('spanish'))
stop3 = set(stopwords.words('french'))
stop=[stop1,stop2,stop3]
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
str_abstract=[str(abstract) for abstract in df['abstract']]
abstract_array =[clean(abstract).split() for abstract in str_abstract]


In [44]:
#join the cleaned abstracts
abstract_array_joined =[" ".join(clean(abstract).split()) for abstract in str_abstract]
    

In [45]:
sentences = abstract_array
new_sent=[]
for i in sentences:
    new_sent.append(str(i))
sentences = ["[SEP]"+sentence +"[CLS]" for sentence in new_sent]

In [46]:
#research XLNet a little more, make sure implemntation is correct
tokenizer_2 = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)
tokenized_texts_2 = [tokenizer_2.tokenize(sent) for sent in sentences]
print(tokenized_texts_2[0])

['▁[', 's', 'ep', ']', '[', "'", 'next', 'generation', "'", ',', '▁', "'", 'se', 'qu', 'encing', "'", ',', '▁', "'", 'is', "'", ',', '▁', "'", 'increasing', 'ly', "'", ',', '▁', "'", 'being', "'", ',', '▁', "'", 'used', "'", ',', '▁', "'", 'to', "'", ',', '▁', "'", 'stud', 'y', "'", ',', '▁', "'", 's', 'amp', 'le', "'", ',', '▁', "'", 'com', 'posed', "'", ',', '▁', "'", 'of', "'", ',', '▁', "'", 'mix', 'ture', "'", ',', '▁', "'", 'of', "'", ',', '▁', "'", 'organ', 'ism', "'", ',', '▁', "'", 'such', "'", ',', '▁', "'", 'a', "'", ',', '▁', "'", 'in', "'", ',', '▁', "'", 'clin', 'ical', "'", ',', '▁', "'", 'application', "'", ',', '▁', "'", 'where', "'", ',', '▁', "'", 'the', "'", ',', '▁', "'", 'pre', 's', 'ence', "'", ',', '▁', "'", 'of', "'", ',', '▁', "'", 'a', "'", ',', '▁', "'", 'path', 'ogen', "'", ',', '▁', "'", 'at', "'", ',', '▁', "'", 'very', "'", ',', '▁', "'", 'low', "'", ',', '▁', "'", 'a', 'bund', 'ance', "'", ',', '▁', "'", 'may', "'", ',', '▁', "'", 'be', "'", ',', '▁', "

In [47]:
input_ids_2 = [tokenizer_2.convert_tokens_to_ids(x) for x in tokenized_texts_2]

In [48]:
MAX_LEN = 150

In [49]:
input_ids_2 = pad_sequences(input_ids_2, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [50]:
print(input_ids_2[2])

[ 4145    23  3882  3158 10849    26  2371    98   814    26    19    17
    26  8588   773    26    19    17    26   627    26    19    17    26
 26729  7061 12801 22148    26    19    17    26 14466    26    19    17
    26  3033    26    19    17    26 12579   415    26    19    17    26
   590    26    19    17    26 18479    26    19    17    26  1383    26
    19    17    26  9082 11406 24353    26    19    17    26 26729   769
    26    19    17    26  2438    26    19    17    26 17283    56    26
    19    17    26 16258    26    19    17    26 25026    26    19    17
    26 30906    26    19    17    26   153    26    19    17    26 24617
    26    19    17    26 23467    26    19    17    26   590    26    19
    17    26 31664 11809    26    19    17    26 11730    26    19    17
    26  1168    26    19    17    26    23  4123 18898    26    19    17
    26 27424    26    19    17    26]


In [ ]:
# Dimensionality reduction PCA UMAP TSNE 

In [14]:
#simple kmeans clustering 
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(input_ids_2)


In [15]:
len(kmeans.labels_)

45774

In [16]:
# hierarchal clustering 
from sklearn.cluster import AgglomerativeClustering
agg=AgglomerativeClustering(n_clusters=10).fit(input_ids_2)

In [17]:
agg.labels_

array([1, 0, 3, ..., 4, 7, 1], dtype=int64)

In [18]:
#Clustering algorithm 3
from sklearn.cluster import DBSCAN
db=DBSCAN(eps=3).fit(input_ids_2)


In [ ]:
#Gaussian Mixture Modeling 

In [19]:
db.labels_

array([-1, -1, -1, ..., -1,  0, -1], dtype=int64)

In [20]:
#create a new dataframe
d = {'id':df['cord_uid'] , 'cluster': agg.labels_,'abstract':abstract_array_joined}
clusters=pd.DataFrame(data=d)

In [21]:
clusters.head()

,id,cluster,abstract
0,vho70jcx,1,nextgeneration sequencing is increasingly bein...
1,i9tbix2v,0,an emerging disease is one infectious epidemic...
2,62gfisc6,3,germline variation at immunoglobulin gene ig l...
3,058r9486,0,deep sequencing of clinical sample is now an e...
4,wich35l7,3,developing method to reconstruct transmission ...


In [22]:
gf=clusters.groupby('cluster').count()

In [23]:
gf
#try a double dbscan, dbscan once, then dbscan the first cluster

,id,abstract
cluster,,
0,10660,10660
1,3181,3181
2,1189,1189
3,8353,8353
4,5915,5915
5,2970,2970
6,943,943
7,8199,8199
8,1716,1716


In [24]:
cluster1=clusters[clusters['cluster']==-1].dropna()
cluster2=clusters[clusters['cluster']==0].dropna()
cluster3=clusters[clusters['cluster']==1].dropna()
cluster4=clusters[clusters['cluster']==2].dropna()
cluster5=clusters[clusters['cluster']==3].dropna()

In [25]:
#seperate the clusters into their own dataframes
cluster1=clusters[clusters['cluster']==0].dropna()
cluster2=clusters[clusters['cluster']==1].dropna()
cluster3=clusters[clusters['cluster']==2].dropna()
cluster4=clusters[clusters['cluster']==3].dropna()
cluster5=clusters[clusters['cluster']==4].dropna()
cluster6=clusters[clusters['cluster']==5].dropna()
cluster7=clusters[clusters['cluster']==6].dropna()
cluster8=clusters[clusters['cluster']==7].dropna()
cluster9=clusters[clusters['cluster']==8].dropna()
cluster10=clusters[clusters['cluster']==9].dropna()

In [34]:
#find the frequency of topics 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(cluster7['abstract'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(cluster7['abstract'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [35]:
#topic model
#always more types of topic modeling Latent Discriminate Analysis 
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation( max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [36]:
#display topics
#look up methods for word patterns and frequncy analysis 
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

no_top_words = 10
print_top_words(nmf, tfidf_feature_names, no_top_words)
print_top_words(lda, tf_feature_names, no_top_words)

Topic #0: respiratory child sample infection wa assay virus detected pathogen detection
Topic #1: rna sequence genome viral gene virus replication region dna coronavirus
Topic #2: health disease outbreak infectious public infection risk case ha data
Topic #3: cell mouse infection expression immune receptor viral response cytokine virus
Topic #4: protein membrane domain fusion peptide activity lipid structure cellular binding
Topic #5: antibody monoclonal neutralizing epitope serum igg antigen assay produced used
Topic #6: virus influenza human h5n1 avian h1n1 pathogenic acid pandemic detection
Topic #7: sars sarscov 2003 severe syndrome cov acute respiratory woman kong
Topic #8: vaccine immune vaccination response safety development antigen hajj efficacy strategy
Topic #9: patient group pneumonia day wa covid19 treatment case symptom hospital

Topic #0: sars le la et cov en unit agent 2003 epidemic
Topic #1: vaccine disease immune response study vaccination milk review abstract ha
Topi